# 데이터 전처리

In [80]:
labels = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
data_path = '.' 

from glob import glob
import random
import os
import numpy as np

SEED = 2018

# 리스트를 랜덤하게 셔플하는 함수이다
def random_shuffle(lst):
    random.seed(SEED)
    random.shuffle(lst)
    return lst

# 텍스트 파일을 저장할 폴더를 생성한다.
if not os.path.exists('input'):
    os.mkdir('input')

# 훈련 데이터 전체를 먼저 trn_all.txt에 저장한다
trn_all = []
trn_all_file = open('input/trn_all.txt', 'w')
# 제공된 훈련 데이터 경로를 모두 읽어온다
files = glob(os.path.join(os.getcwd(), 'train/train/audio/*/*.wav'))
for f in files:
    # 배경 소음은 skip한다
    if '_background_noise_' in f:
        continue

    # 정답값(label)과 화자(speaker)정보를 파일명에서 추출한다
    label = f.split('/')[-1].split('\\')[1]
    speaker = f.split('/')[-1].split('\\')[-1].split('_')[0]
    if label not in labels:
        # 10개의 label외 데이터는 20%의 확률로 unknown으로 분류하여 추가한다
        label = 'unknown'
        if random.random() < 0.2:
            trn_all.append((label, speaker, f))
            trn_all_file.write('{},{},{}\n'.format(label, speaker, f))
    else:
        trn_all.append((label, speaker, f))
        trn_all_file.write('{},{},{}\n'.format(label, speaker, f))
trn_all_file.close()

In [82]:
# 훈련 데이터를 화자 기반 9:1 비율로 분리한다
uniq_speakers = list(set([speaker for (label, speaker, path) in trn_all]))
random_shuffle(uniq_speakers)
cutoff = int(len(uniq_speakers) * 0.9)
speaker_val = uniq_speakers[cutoff:]

# 교차 검증용 파일을 생성한다
trn_file = open('input/trn.txt', 'w')
val_file = open('input/val.txt', 'w')
for (label, speaker, path) in trn_all:
    if speaker not in speaker_val:
        trn_file.write('{},{},{}\n'.format(label, speaker, path))
    else:
        val_file.write('{},{},{}\n'.format(label, speaker, path))
trn_file.close()
val_file.close()

# 데이터 전처리 루틴 구현

In [83]:
import torch
import numpy as np
from torch.utils.data import Dataset
import librosa
from glob import glob
import random

# 음성 파일의 sample rate은 1초 = 16000으로 지정한다
SR = 16000

# 경진대회 전용 torch SpeechDataset 클래스를 정의한다
class SpeechDataset(Dataset):
    def __init__(self, mode, label_to_int, wav_list, label_list=None):
        self.mode = mode
        self.label_to_int = label_to_int
        self.wav_list = wav_list
        self.label_list = label_list
        self.sr = SR
        self.n_silence = int(len(wav_list) * 0.1)

        # 배경 소음 데이터를 미리 읽어온다
        self.background_noises = [librosa.load(x, sr=self.sr)[0] for x in glob("./train/train/audio/_background_noise_/*.wav")]

    def get_one_word_wav(self, idx):
        # idx 번째 음성 파일을 1초만큼 읽어온다
        wav = librosa.load(self.wav_list[idx], sr=self.sr)[0]
        if len(wav) < self.sr:
            wav = np.pad(wav, (0, self.sr - len(wav)), 'constant')
        return wav[:self.sr]

    def get_one_noise(self):
        # 배경 소음 데이터 중 랜덤하게 1초를 읽어온다
        selected_noise = self.background_noises[random.randint(0, len(self.background_noises) - 1)]
        start_idx = random.randint(0, len(selected_noise) - 1 - self.sr)
        return selected_noise[start_idx:(start_idx + self.sr)]

    def get_mix_noises(self, num_noise=1, max_ratio=0.1):
        # num_noise 만큼의 배경 소음을 합성한다
        result = np.zeros(self.sr)
        for _ in range(num_noise):
            result += random.random() * max_ratio * self.get_one_noise()
        return result / num_noise if num_noise > 0 else result

    def get_silent_wav(self, num_noise=1, max_ratio=0.5):
        # 배경 소음 데이터를 silence로 가정하고 불러온다
        return self.get_mix_noises(num_noise=num_noise, max_ratio=max_ratio)

    def __len__(self):
        # 교차검증 모드일 경우에는 ‘silence’를 추가한 만큼이 데이터 크기이고, Test 모드일 경우에는 제공된 테스트 데이터가 전부이다
        if self.mode == 'test':
            return len(self.wav_list)
        else:
            return len(self.wav_list) + self.n_silence

    def __getitem__(self, idx):
        # idx번째 음성 데이터 하나를 반환한다
        if idx < len(self.wav_list):
            # 전처리는 mel spectrogram으로 지정한다
            # (옵션) 여기서 Data Augmentation을 수행할 수 있다.
            wav_numpy = preprocess_mel(self.get_one_word_wav(idx))
            wav_tensor = torch.from_numpy(wav_numpy).float()
            wav_tensor = wav_tensor.unsqueeze(0)

            # 음성 스펙트로그램(spec), 파일 경로(id)와 정답값(label)을 반환한다
            if self.mode == 'test':
                return {'spec': wav_tensor, 'id': self.wav_list[idx]}
            else:
                label = self.label_to_int.get(self.label_list[idx], len(self.label_to_int))
                return {'spec': wav_tensor, 'id': self.wav_list[idx], 'label': label}
        else:
            # 배경 소음을 반환한다
            wav_numpy = preprocess_mel(self.get_silent_wav(
                num_noise=random.choice([0, 1, 2, 3]),
                max_ratio=random.choice([x / 10. for x in range(20)])))
            wav_tensor = torch.from_numpy(wav_numpy).float()
            wav_tensor = wav_tensor.unsqueeze(0)
            return {'spec': wav_tensor, 'id': 'silence', 'label': len(self.label_to_int) + 1}

# mel spectrogram 전처리 함수이다
def preprocess_mel(data, n_mels=40):
    spectrogram = librosa.feature.melspectrogram(data, sr=SR, n_mels=n_mels, hop_length=160, n_fft=480, fmin=20, fmax=4000)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

# 모델 구현

In [84]:
import torch
import torch.nn.functional as F
from torch.nn import MaxPool2d

# ResNet 모델을 구현한다
class ResModel(torch.nn.Module):
    def __init__(self):
        super(ResModel, self).__init__()
        # 이번 경진대회에 사용되는 label 개수는 12이다
        n_labels = 12
        n_maps = 128
        # 총 9계층 모델을 쌓는다
        self.n_layers = n_layers = 9
        # 첫 계층에 사용하는 convolutional 모듈을 정의한다
        self.conv0 = torch.nn.Conv2d(1, n_maps, (3, 3), padding=(1, 1), bias=False)
        # MaxPooling 모듈을 정의한다
        self.pool = MaxPool2d(2, return_indices=True)
        # 2계층 이후에 사용하는 convolutional 모듈을 정의한다
        self.convs = torch.nn.ModuleList([torch.nn.Conv2d(n_maps, n_maps, (3, 3), padding=1, dilation=1, bias=False) for _ in range(n_layers)])
        # BatchNormalization 모듈과 conv 모듈을 조합한다
        for i, conv in enumerate(self.convs):
            self.add_module("bn{}".format(i + 1), torch.nn.BatchNorm2d(n_maps, affine=False))
            self.add_module("conv{}".format(i + 1), conv)
        # 최종 계층에는 선형 모듈을 추가한다
        self.output = torch.nn.Linear(n_maps, n_labels)

    def forward(self, x):
        for i in range(self.n_layers + 1):
            y = F.relu(getattr(self, "conv{}".format(i))(x))
            if i == 0:
                old_x = y
            # 이전 layer의 결과값(old_x)와 이번 layer 결과값(y)을 더하는 것이 residual 모듈이다 
            if i > 0 and i % 2 == 0:
                x = y + old_x
                old_x = x
            else:
                x = y
            # BatchNormalization을 통해 파라미터 값을 정규화한다
            if i > 0:
                x = getattr(self, "bn{}".format(i))(x)
            # pooling을 사용할지 True/False로 지정한다
            pooling = False
            if pooling:
                x_pool, pool_indices = self.pool(x)
                x = self.unpool(x_pool, pool_indices, output_size=x.size())
        x = x.view(x.size(0), x.size(1), -1)
        x = torch.mean(x, 2)
        # 최종 선형 계층을 통과한 결과값을 반환한다
        return self.output(x)

# pipline

In [85]:
"""
model trainer
"""
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch
from time import time
from torch.nn import Softmax
import numpy as np
import pandas as pd
import os
from random import choice
from tqdm import tqdm

# 파일 생성 함수
def create_directory(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

# 시간 체크
def get_time(now, start):
    time_in_min = int((now - start) / 60)
    return time_in_min

# 학습을 위한 기본 설정값을 지정한다
BATCH_SIZE = 32  # 데이터 묶음에 해당하는 batch_size는 GPU 메모리에 알맞게 지정한다
mGPU = False  # multi-GPU를 사용할 경우에는 True로 지정한다
epochs = 20  # 모델이 훈련 데이터를 학습하는 횟수를 지정한다
mode = 'cv' # 교차 검증 모드(cv) or 테스트 모드(test)
model_name = os.path.join(os.getcwd(), 'model/model_resnet.pth' )# 모델 결과물을 저장할 때 모델 이름을 지정한다

# ResNet 모델을 활성화한다
loss_fn = torch.nn.CrossEntropyLoss()
model = ResModel
speechmodel = torch.nn.DataParallel(model()) if mGPU else model()
speechmodel = speechmodel.cuda()

# SpeechDataset을 활성화한다
labels = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
label_to_int = dict(zip(labels, range(len(labels))))
int_to_label = dict(zip(range(len(labels)), labels))
int_to_label.update({len(labels): 'unknown', len(labels) + 1: 'silence'})

# 모드에 따라 학습 및 검증에 사용할 파일을 선택한다
trn = 'input/trn.txt' if mode == 'cv' else 'input/trn_all.txt'
tst = 'input/val.txt' if mode == 'cv' else 'input/tst.txt'

trn = [line.strip() for line in open(trn, 'r').readlines()]
wav_list = [line.split(',')[-1] for line in trn]
label_list = [line.split(',')[0] for line in trn]
# 학습용 SpeechDataset을 불러온다
traindataset = SpeechDataset(mode='train', label_to_int=label_to_int, wav_list=wav_list, label_list=label_list)

start_time = time()
for e in range(epochs):
    print("training epoch ", e)
    # learning_rate를 epoch마다 다르게 지정한다
    learning_rate = 0.01 if e < 10 else 0.001
    optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, speechmodel.parameters()), lr=learning_rate, momentum=0.9, weight_decay=0.00001)
    # 모델을 학습하기 위하여 .train() 함수를 실행한다
    speechmodel.train()

    total_correct = 0
    num_labels = 0
    trainloader = DataLoader(traindataset, BATCH_SIZE, shuffle=True)
    # 학습을 수행한다
    for batch_idx, batch_data in enumerate(tqdm(trainloader)):
        speechmodel.train()
        # batch_size 만큼의 음성 데이터(spec)와 정답값(label)을 받아온다
        spec = batch_data['spec']
        label = batch_data['label']
        spec, label = Variable(spec.cuda()), Variable(label.cuda())
        # 현재 모델의 예측값(y_pred)을 계산한다
        y_pred = speechmodel(spec)
        _, pred_labels = torch.max(y_pred.data, 1)
        correct = (pred_labels == label.data).sum()
        # 정답과 예측값간의 차이(loss)를 계산한다 
        loss = loss_fn(y_pred, label)

        total_correct += correct
        num_labels += len(label)
    
        optimizer.zero_grad()
        # loss를 기반으로 back-propagation을 수행한다
        loss.backward()
        # 모델 파라미터를 업데이트한다. (실질적 학습)
        optimizer.step()
    
    # 훈련 데이터에서의 정확률을 기록한다
    print("training accuracy:", 100. * total_correct / num_labels, get_time(time(), start_time))

    # 교차 검증 모드의 경우, 검증 데이터에 대한 정확률을 기록한다
    if mode == 'cv':
        # 현재 학습 중인 모델을 임시로 저장한다
        torch.save(speechmodel.state_dict(), '{}_cv'.format(model_name))
        
        # 검증 데이터를 불러온다
        softmax = Softmax()
        tst_list = [line.strip() for line in open(tst, 'r').readlines()]
        wav_list = [line.split(',')[-1] for line in tst_list]
        label_list = [line.split(',')[0] for line in tst_list]
        cvdataset = SpeechDataset(mode='test', label_to_int=label_to_int, wav_list=wav_list)
        cvloader = DataLoader(cvdataset, BATCH_SIZE, shuffle=False)

        # 모델을 불러와 .eval() 함수로 검증 준비를 한다
        speechmodel = torch.nn.DataParallel(model()) if mGPU else model()
        speechmodel.load_state_dict(torch.load('{}_cv'.format(model_name)))
        speechmodel = speechmodel.cuda()
        speechmodel.eval()

        # 검증 데이터를 batch_size만큼씩 받아오며 예측값을 저장한다
        fnames, preds = [], []
        for batch_idx, batch_data in enumerate(tqdm(cvloader)):
            spec = Variable(batch_data['spec'].cuda())
            fname = batch_data['id']
            y_pred = softmax(speechmodel(spec))
            preds.append(y_pred.data.cpu().numpy())
            fnames += fname

        preds = np.vstack(preds)
        preds = [int_to_label[x] for x in np.argmax(preds, 1)]
        fnames = [fname.split('/')[-2] for fname in fnames]
        num_correct = 0
        for true, pred in zip(fnames, preds):
            if true == pred:
                num_correct += 1

        # 검증 데이터의 정확률을 기록한다
        print("cv accuracy:", 100. * num_correct / len(preds), get_time(time(), start_time))

# 학습이 완료된 모델을 저장한다
create_directory("model")
torch.save(speechmodel.state_dict(), model_name)

# # 테스트 데이터에 대한 예측값을 파일에 저장한다
# print("doing prediction...")
# softmax = Softmax()

# # 테스트 데이터를 불러온다
# tst = [line.strip() for line in open(tst, 'r').readlines()]
# wav_list = [line.split(',')[-1] for line in tst]
# testdataset = SpeechDataset(mode='test', label_to_int=label_to_int, wav_list=wav_list)
# testloader = DataLoader(testdataset, BATCH_SIZE, shuffle=False)

# # 모델을 불러온다
# speechmodel = torch.nn.DataParallel(model()) if mGPU else model()
# speechmodel.load_state_dict(torch.load(model_name))
# speechmodel = speechmodel.cuda()
# speechmodel.eval()
    
# test_fnames, test_labels = [], []
# pred_scores = []

# # 테스트 데이터에 대한 예측값을 계산한다
# for batch_idx, batch_data in enumerate(tqdm(testloader)):
#     spec = Variable(batch_data['spec'].cuda())
#     fname = batch_data['id']
#     y_pred = softmax(speechmodel(spec))
#     pred_scores.append(y_pred.data.cpu().numpy())
#     test_fnames += fname

# # 가장 높은 확률값을 가진 예측값을 label 형태로 저장한다
# final_pred = np.vstack(pred_scores)
# final_labels = [int_to_label[x] for x in np.argmax(final_pred, 1)]
# test_fnames = [x.split("/")[-1] for x in test_fnames]

# # 테스트 파일 명과 예측값을 sub 폴더 아래 저장한다. 캐글에 직접 업로드 할 수 있는 파일 포맷이다.
# create_directory("sub")
# pd.DataFrame({'fname': test_fnames, 'label': final_labels}).to_csv("sub/{}.csv".format(model_name.split('/')[-1]), index=False)

training epoch  0


100%|█████████████████████████████████████████| 979/979 [02:23<00:00,  6.83it/s]


training accuracy: tensor(37.3886, device='cuda:0') 2


  0%|                                                   | 0/109 [00:00<?, ?it/s]C:\Users\master\AppData\Local\Temp/ipykernel_30744/4136072718.py:116: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_pred = softmax(speechmodel(spec))
100%|█████████████████████████████████████████| 109/109 [00:15<00:00,  7.01it/s]


cv accuracy: 0.0 2
training epoch  1


...

cv accuracy: 0.0 45
training epoch  19


100%|█████████████████████████████████████████| 979/979 [02:11<00:00,  7.42it/s]


training accuracy: tensor(97.8829, device='cuda:0') 47


100%|█████████████████████████████████████████| 109/109 [00:10<00:00, 10.21it/s]

cv accuracy: 0.0 47
